Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, hour, unix_timestamp, dayofweek
from pyspark.sql.types import *
from pyspark.sql.functions import broadcast
from pyspark.sql import functions as f

VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
0,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2.1. Import Yellow and Green taxi data (unnormalized).

In [2]:
# Function to process Parquet files with schema unification
def process_parquet_files(spark, path: str, target_schema: StructType) -> DataFrame:
    # Read the directory 
    temp_df = spark.read.option("recursiveFileLookup", "true").parquet(path)
    
    # Get the list of all Parquet files 
    files = temp_df.inputFiles()  # Returns a list of full file paths
#     print(f"Found files in {path}:")
#     print(files)

    # Process each file individually
    dataframes = []
    for file in files:
        print(f"Processing file: {file}")
        
        # Read the file 
        df = spark.read.parquet(file)

        # Cast columns to match the target schema
        for field in target_schema:
            column_name = field.name
            column_type = field.dataType.simpleString()  
            
            # Check if the column exists in the file
            if column_name in df.columns:
                df = df.withColumn(column_name, col(column_name).cast(column_type))
            else:
                # Add missing columns with null values, cast to the expected type
                df = df.withColumn(column_name, col(column_name).cast(column_type).alias(column_name))
        
        # Add the unified DataFrame for the file to the list
        dataframes.append(df)

    # Combine all DataFrames
    final_df = dataframes[0]
    for df in dataframes[1:]:
        final_df = final_df.unionByName(df)  # Align columns by name
    
    return final_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'DataFrame' is not defined
Traceback (most recent call last):
NameError: name 'DataFrame' is not defined



In [20]:
# Yellow Taxi data processing
yellow_taxi_path = "s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow"

# Define the Yellow Taxi schema
yellow_target_schema = StructType([
    StructField("VendorID", LongType(), True),
    StructField("tpep_pickup_datetime", TimestampType(), True),
    StructField("tpep_dropoff_datetime", TimestampType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("RatecodeID", LongType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("payment_type", LongType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
    StructField("airport_fee", DoubleType(), True),
])

# Apply unification function to Yellow Taxi data
yellow_taxi_df = process_parquet_files(spark, yellow_taxi_path, yellow_target_schema)

# Check the results
# yellow_taxi_df.show(5)
yellow_taxi_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2020/yellow_tripdata_2020-01.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2017/yellow_tripdata_2017-01.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2015/yellow_tripdata_2015-09.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2020/yellow_tripdata_2020-12.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2022/yellow_tripdata_2022-05.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2018/yellow_tripdata_2018-12.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2018/yellow_tripdata_2018-04.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2018/yellow_tripdata_2018-09.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/yellow/2014/yellow_tripdata_2014-08.

In [21]:
# Green Taxi data processing
green_taxi_path = "s3://robot-dreams-source-data/home-work-1/nyc_taxi/green"

# Define the Green Taxi schema
green_target_schema = StructType([
    StructField("VendorID", LongType(), True),
    StructField("lpep_pickup_datetime", TimestampType(), True),
    StructField("lpep_dropoff_datetime", TimestampType(), True),
    StructField("store_and_fwd_flag", StringType(), True),
    StructField("RatecodeID", LongType(), True),
    StructField("PULocationID", LongType(), True),
    StructField("DOLocationID", LongType(), True),
    StructField("passenger_count", LongType(), True),
    StructField("trip_distance", DoubleType(), True),
    StructField("fare_amount", DoubleType(), True),
    StructField("extra", DoubleType(), True),
    StructField("mta_tax", DoubleType(), True),
    StructField("tip_amount", DoubleType(), True),
    StructField("tolls_amount", DoubleType(), True),
    StructField("ehail_fee", DoubleType(), True),
    StructField("improvement_surcharge", DoubleType(), True),
    StructField("total_amount", DoubleType(), True),
    StructField("payment_type", LongType(), True),
    StructField("trip_type", DoubleType(), True),
    StructField("congestion_surcharge", DoubleType(), True),
])

# Apply unification function to Green Taxi data
green_taxi_df = process_parquet_files(spark, green_taxi_path, green_target_schema)

# Check the results
# green_taxi_df.show(5)
green_taxi_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2015/green_tripdata_2015-09.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2015/green_tripdata_2015-06.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2019/green_tripdata_2019-08.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2021/green_tripdata_2021-11.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2022/green_tripdata_2022-09.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2018/green_tripdata_2018-06.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2019/green_tripdata_2019-02.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2015/green_tripdata_2015-03.parquet
Processing file: s3://robot-dreams-source-data/home-work-1/nyc_taxi/green/2022/green_tripdata_2022-05.parquet
Processing

In [ ]:
# Check that data was indeed unified
green_taxi_df.filter("congestion_surcharge > 0").show()
# yellow_taxi_df.filter("congestion_surcharge > 0").show()

### 2.2. Unify schemas: Make sure all columns have the same names and data types. Add a 'taxi_type' column with the appropriate values: yellow or green.

In [23]:
# Unify columns and add 'taxi_type' column
green_taxi_df = green_taxi_df \
    .withColumn("taxi_type", lit("green")) \
    .withColumnRenamed("lpep_pickup_datetime", "pickup_datetime") \
    .withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")

yellow_taxi_df = yellow_taxi_df \
    .withColumn("taxi_type", lit("yellow")) \
    .withColumnRenamed("tpep_pickup_datetime", "pickup_datetime") \
    .withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2.3. Combine the data into a single raw_trips_df dataframe.

In [25]:
# Union the data
raw_trips_df = yellow_taxi_df.unionByName(green_taxi_df, allowMissingColumns = True)

# Check the schema
print("United dataframe schema:")
raw_trips_df.printSchema()

# Show example
raw_trips_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

United dataframe schema:
root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- taxi_type: string (nullable = false)
 |-- ehail_fee: double (nullable = true)
 |-- trip_type: double (nullable = true)

+--

In [27]:
# Check number of partitions 
# raw_trips_df.rdd.getNumPartitions() 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Cache raw_trips_df data for better perfomance  
raw_trips_df.cache()

# Data check to test union 
raw_trips_df.count() # + trigger catching as an action

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Yellow Taxi Count: 745067811, Green Taxi Count: 82630053, Raw Trips Count: 827697864
Check passed

### 2.4. Filter out anomalous records: Remove trips with distance < 0.1 km, fare < 2, duration < 1 min.

In [30]:
# Define filter conditions for valid data
valid_condition = (
    (col("trip_distance") >= 0.1) &  # trips with distance >= 0.1 km
    (col("fare_amount") >= 2) &      # trips with fare amount >= $2
    ((unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))) >= 60)  # trips with duration >= 1 minute
)

# Filter valid trips (normal trips)
filtered_trips_df = raw_trips_df.filter(valid_condition)

# Filtering abnormal trips 
abnormal_trips_df = raw_trips_df.filter(~valid_condition)  

# Save abnormal trips to storage, so malformed data can be analyzed later ('overwrite' mode was choosen just for usability)
abnormal_trips_df.write.mode("overwrite").parquet("s3://havdeeva-emr-studio/data/stage/abnormal_data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# Show example of filtered data
filtered_trips_df.select("VendorID", "trip_distance", "fare_amount").orderBy("trip_distance","fare_amount").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------+-----------+
|VendorID|trip_distance|fare_amount|
+--------+-------------+-----------+
|       1|          0.1|        2.0|
|       2|          0.1|        2.0|
|       2|          0.1|        2.0|
|       1|          0.1|        2.0|
|       1|          0.1|        2.0|
+--------+-------------+-----------+
only showing top 5 rows

In [32]:
# Show example of abnormal data
abnormal_trips_df.select("VendorID", "trip_distance", "fare_amount").orderBy("trip_distance","fare_amount").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------+-----------+
|VendorID|trip_distance|fare_amount|
+--------+-------------+-----------+
|       1|-4.08401244E7|        2.5|
|       1|-4.00955322E7|        0.0|
|       1|   -3446729.6|        2.5|
|       1|    -186318.4|        2.5|
|       2|    -20329.08|      40.45|
+--------+-------------+-----------+
only showing top 5 rows

### 2.5. Add the following columns: pickup_hour — the hour the trip starts, pickup_day_of_week — the day of the week, duration_min — the duration of the trip in minutes.

In [33]:
# Add new columns to the filtered dataset 
new_trips_df = filtered_trips_df.withColumn(
    "pickup_hour", hour(col("pickup_datetime"))  # Extract pickup hour from timestamp
).withColumn(
    "pickup_day_of_week", dayofweek(col("pickup_datetime"))  # Extract pickup day of week 
).withColumn(
    "duration_min", (unix_timestamp(col("dropoff_datetime")) - unix_timestamp(col("pickup_datetime"))) / 60  # Calculate duration in minutes
)

print("Yellow taxi DataFrame schema:")
new_trips_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Yellow taxi DataFrame schema:
root
 |-- VendorID: long (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- taxi_type: string (nullable = false)
 |-- ehail_fee: double (nullable = true)
 |-- trip_type: double (nullable = true)

In [34]:
# Show example of added data
new_trips_df.select("VendorID", "pickup_datetime", "pickup_hour", "pickup_datetime", "pickup_day_of_week", "dropoff_datetime", "pickup_datetime", "duration_min").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------+-----------+-------------------+------------------+-------------------+-------------------+-----------------+
|VendorID|    pickup_datetime|pickup_hour|    pickup_datetime|pickup_day_of_week|   dropoff_datetime|    pickup_datetime|     duration_min|
+--------+-------------------+-----------+-------------------+------------------+-------------------+-------------------+-----------------+
|       1|2020-01-01 00:28:15|          0|2020-01-01 00:28:15|                 4|2020-01-01 00:33:03|2020-01-01 00:28:15|              4.8|
|       1|2020-01-01 00:35:39|          0|2020-01-01 00:35:39|                 4|2020-01-01 00:43:04|2020-01-01 00:35:39|7.416666666666667|
|       1|2020-01-01 00:47:41|          0|2020-01-01 00:47:41|                 4|2020-01-01 00:53:52|2020-01-01 00:47:41|6.183333333333334|
|       1|2020-01-01 00:55:23|          0|2020-01-01 00:55:23|                 4|2020-01-01 01:00:14|2020-01-01 00:55:23|             4.85|
|       1|2020-01-01

### 2.6. Perform a JOIN with taxi_zone_lookup.csv, adding the pickup_zone, dropoff_zone fields.

In [35]:
# Define taxi_zone_lookup.csv path
TAXI_ZONE_LOOKUP_PATH = "s3://robot-dreams-source-data/home-work-1-unified/nyc_taxi/taxi_zone_lookup.csv"

# Read taxi_zone_lookup data
taxi_zone_lookup_df = spark.read.option("header", "true").csv(TAXI_ZONE_LOOKUP_PATH)

# Add new columns in advance for better JOIN performance
taxi_zone_lookup_prepared_df = taxi_zone_lookup_df \
    .withColumn("PULocationID", col("LocationID")) \
    .withColumn("DOLocationID", col("LocationID"))

print("Taxi zone lookup DataFrame schema:")
taxi_zone_lookup_df.printSchema()

# Perform JOIN for both pickup_zone and dropoff_zone
final_stage_df = new_trips_df \
    .join(
        broadcast(taxi_zone_lookup_prepared_df.alias("pickup")), 
        new_trips_df["PULocationID"] == col("pickup.PULocationID"),
        how="left"
    ) \
    .withColumnRenamed("Zone", "pickup_zone") \
    .drop("PULocationID") \
    .join(
        broadcast(taxi_zone_lookup_prepared_df.alias("dropoff")), 
        new_trips_df["DOLocationID"] == col("dropoff.DOLocationID"),
        how="left"
    ) \
    .withColumnRenamed("Zone", "dropoff_zone")

# Show the final DataFrame
final_stage_df.select("VendorID", "PULocationID", "pickup_zone", "DOLocationID", "dropoff_zone").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Taxi zone lookup DataFrame schema:
root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

+--------+------------+--------------------+------------+--------------------+
|VendorID|PULocationID|         pickup_zone|DOLocationID|        dropoff_zone|
+--------+------------+--------------------+------------+--------------------+
|       1|         238|Upper West Side N...|         239|Upper West Side S...|
|       1|         239|Upper West Side S...|         238|Upper West Side N...|
|       1|         238|Upper West Side N...|         238|Upper West Side N...|
|       1|         238|Upper West Side N...|         151|    Manhattan Valley|
|       1|         246|West Chelsea/Huds...|          48|        Clinton East|
+--------+------------+--------------------+------------+--------------------+
only showing top 5 rows

In [36]:
# Save stage data to storage ('overwrite' mode was choosen just for usability)
final_stage_df.write.mode("overwrite").parquet("s3://havdeeva-emr-studio/data/stage/final_data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 3.1. Creating and aggregating the final zone_summary dataframe

In [42]:
# Group the DataFrame by pickup_zone and perform aggregations
zone_summary_df = final_stage_df.groupBy("pickup_zone").agg(
    f.count("*").alias("total_trips"),                               # Total trips
    f.avg("trip_distance").alias("avg_trip_distance"),               # Average distance
    f.avg("total_amount").alias("avg_total_amount"),                 # Average total amount
    f.avg("tip_amount").alias("avg_tip_amount"),                     # Average tip amount
    f.max("trip_distance").alias("max_trip_distance"),               # Maximum trip distance
    f.min("tip_amount").alias("min_tip_amount"),                     # Minimum tip amount
    f.sum(f.when(col("taxi_type") == "yellow", 1).otherwise(0))      # Count yellow trips
        .alias("yellow_trips"),
    f.sum(f.when(col("taxi_type") == "green", 1).otherwise(0))       # Count green trips
        .alias("green_trips")
)

# Compute yellow_share and green_share
zone_summary_df = zone_summary_df.withColumn(
    "yellow_share", (f.col("yellow_trips") / f.col("total_trips")) * 100
).withColumn(
    "green_share", (f.col("green_trips") / f.col("total_trips")) * 100
).drop("yellow_trips", "green_trips")  # Drop intermediate counts

# Show the resulting zone_summary DataFrame
zone_summary_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+------------------+------------------+-------------------+-----------------+--------------+------------------+--------------------+
|         pickup_zone|total_trips| avg_trip_distance|  avg_total_amount|     avg_tip_amount|max_trip_distance|min_tip_amount|      yellow_share|         green_share|
+--------------------+-----------+------------------+------------------+-------------------+-----------------+--------------+------------------+--------------------+
|         Westerleigh|       1246| 8.713531300160511| 39.15078651685394|  2.402479935794543|            116.8|           0.0| 48.23434991974318|   51.76565008025682|
|Charleston/Totten...|       1746|28.525727376861415|  89.6182932416953| 2.3222909507445593|            103.0|           0.0|  87.0561282932417|  12.943871706758307|
|      Pelham Parkway|      97553|16.481890562053447|18.983224401094677| 0.5783059465111277|        221944.04|           0.0|10.126802866134305|   89.87319713386569|
|   

### 4. Aggregation by day of the week and zone


### 4.1. Count the number of trips for a given day of the week (e.g., Monday — Sunday, etc.).
### 4.2. Calculate the share of trips with fares over $30 for each zone (high_fare_share).

In [47]:
# Group by pickup_zone and pickup_day_of_week, compute aggregations
zone_days_summary_df = final_stage_df.groupBy("pickup_zone", "pickup_day_of_week").agg(
    f.count("*").alias("total_trips"),                           # Total trips per zone and day
    f.sum(f.when(f.col("total_amount") > 30, 1).otherwise(0))    # Total trips with fare > $30
        .alias("high_fare_trips")
)

# Calculate high_fare_share (percentage of trips with fare > $30)
zone_days_summary_df = zone_days_summary_df.withColumn(
    "high_fare_share", (f.col("high_fare_trips") / f.col("total_trips")) * 100
).drop("high_fare_trips")  # Drop intermediate column

# Show the result
zone_days_summary_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+-----------+------------------+
|         pickup_zone|pickup_day_of_week|total_trips|   high_fare_share|
+--------------------+------------------+-----------+------------------+
|Washington Height...|                 6|     111674|15.630316815015133|
|         Old Astoria|                 7|     176034| 9.358987468329982|
|           Hudson Sq|                 5|     682825| 8.317797385860214|
|                SoHo|                 1|     919281| 5.897543841328169|
|       South Jamaica|                 5|      14183| 69.22371853627583|
|Marine Park/Mill ...|                 5|       8735| 33.87521465369204|
|        Bedford Park|                 2|      23499|10.208945061491978|
|            Longwood|                 4|      10177|13.402770954112214|
|     Carroll Gardens|                 5|     153368|14.619086119659903|
|            Gramercy|                 2|    2125183| 4.826266726206637|
|          Ocean Hill|                 3|      1872

### 3.2, 4.3. Save aggregation results: Write the results to a single files

In [58]:
import datetime

# Format date as YYYYMMDD
current_date = datetime.datetime.now().strftime("%Y%m%d")

# Save the zone_days_stats DataFrame to S3 in Parquet format
zone_days_summary_df.coalesce(1).write.mode("overwrite").parquet(
    f"s3://havdeeva-emr-studio/results/zone_statistic/{current_date}/zone_statistic.parquet"
)
zone_summary_df.coalesce(1).write.mode("overwrite").parquet(
    f"s3://havdeeva-emr-studio/results/zone_days_statistic/{current_date}/zone_days_statistic.parquet"
)

print("Data successfully written to bucket")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data successfully written to bucket